Corroborar si este es mas rapido al no seleccionar días separados

In [1]:
# import os
# import sqlite3
# import pandas as pd
# import cdsapi
# import zipfile
# import tempfile
# from netCDF4 import Dataset, num2date

# # ————— Configuración de prototipo —————
# DB_PATH   = r"C:\Users\Nainh\Proton Drive\nainho1306\My files\Licitacion\Plataforma local\backend\data\mi_base_de_datos5.db"
# TMP_DIR   = "tmp_test_inc"
# CDS_URL   = 'https://cds.climate.copernicus.eu/api'
# CDS_KEY   = 'd9af180c-f7f8-4a2e-8029-09018fb8c920'
# AREA      = [-32.0, -63.0, -33.0, -61.0]
# VARIABLES = ['2m_temperature','2m_dewpoint_temperature','total_precipitation']

# os.makedirs(TMP_DIR, exist_ok=True)

# # ————— Conectar y asegurar tabla —————
# conn = sqlite3.connect(DB_PATH)
# cur  = conn.cursor()
# cur.execute("""
# CREATE TABLE IF NOT EXISTS climate_test (
#     date TEXT,
#     latitude REAL, longitude REAL,
#     t2m REAL, d2m REAL, tp REAL,
#     PRIMARY KEY(date, latitude, longitude)
# )
# """)
# conn.commit()

# # ————— Determinar rango de fechas faltantes —————
# # 1) Consultar la última fecha
# cur.execute("SELECT MAX(date) FROM climate_test")
# last = cur.fetchone()[0]  # e.g. '2025-06-15' o None

# if last is None:
#     # si no hay datos, arrancar 6 meses atrás
#     start = pd.to_datetime("today").normalize() - pd.DateOffset(months=6)
# else:
#     # arrancar un día después de la última fecha
#     start = pd.to_datetime(last) + pd.Timedelta(days=1)

# # hasta ayer
# end = pd.to_datetime("today").normalize() - pd.Timedelta(days=1)

# if start > end:
#     print("La base ya está al día (hasta", end.date(), ")")
#     conn.close()
#     exit()

# dates = pd.date_range(start, end, freq="D")
# print(f"Faltan datos de {start.date()} a {end.date()} ({len(dates)} días)")

# # ————— Función de ingesta (idéntica) —————
# def ingest_from_zip(table_name, zip_filepath):
#     extracted = []
#     with zipfile.ZipFile(zip_filepath, 'r') as z:
#         nc_members = [m for m in z.namelist() if m.endswith('.nc')]
#         for member in nc_members:
#             tmpf = tempfile.NamedTemporaryFile(suffix=".nc", delete=False)
#             tmpf.write(z.read(member)); tmpf.close()
#             extracted.append(tmpf.name)

#     # leer dimensiones y variables
#     data = {}
#     times = lats = lons = None
#     for path in extracted:
#         ds = Dataset(path)
#         if times is None:
#             tvar = next(v for v in ds.variables if "time" in v.lower())
#             latv = next(v for v in ds.variables if "lat"  in v.lower())
#             lonv = next(v for v in ds.variables if "lon"  in v.lower())
#             times = num2date(ds.variables[tvar][:],
#                              ds.variables[tvar].units,
#                              getattr(ds.variables[tvar],'calendar','standard'))
#             lats  = ds.variables[latv][:]
#             lons  = ds.variables[lonv][:]
#         # la variable restante
#         vars_ = [v for v in ds.variables if v not in (tvar, latv, lonv, "number")]
#         data[vars_[0]] = ds.variables[vars_[0]][:]
#         ds.close()

#     sql = f"""INSERT OR REPLACE INTO {table_name}
#               (date, latitude, longitude, t2m, d2m, tp)
#               VALUES (?,?,?,?,?,?)"""
#     cnt = 0
#     for ti, dt in enumerate(times):
#         dstr = dt.strftime("%Y-%m-%d")
#         for yi, lat in enumerate(lats):
#             for xi, lon in enumerate(lons):
#                 v2m = float(data.get("2m_temperature", data.get("t2m"))[ti,yi,xi])
#                 vdm = float(data.get("2m_dewpoint_temperature", data.get("d2m"))[ti,yi,xi])
#                 tp  = float(data.get("total_precipitation", data.get("tp"))[ti,yi,xi])
#                 cur.execute(sql, (dstr, float(lat), float(lon), v2m, vdm, tp))
#                 cnt += 1
#     conn.commit()

#     # limpiar
#     for f in extracted + [zip_filepath]:
#         try: os.remove(f)
#         except: pass
#     return cnt

# # ————— Descarga y proceso por mes —————
# c = cdsapi.Client(url=CDS_URL, key=CDS_KEY)
# for period, group in dates.to_series().groupby(dates.to_series().dt.to_period('M')):
#     year  = str(period.year)
#     month = f"{period.month:02d}"
#     days  = [f"{d.day:02d}" for d in group]

#     zip_path = os.path.join(TMP_DIR, f"{year}{month}.zip")
#     print(f"-> {year}-{month}: días {days}")

#     c.retrieve(
#         'derived-era5-single-levels-daily-statistics',
#         {
#             'product_type':    'reanalysis',
#             'format':          'netcdf',
#             'variable':        VARIABLES,
#             'year':            [year],
#             'month':           [month],
#             'day':             days,
#             'daily_statistic': 'daily_mean',
#             'frequency':       '6_hourly',
#             'time_zone':       'utc+00:00',
#             'area':            AREA,
#         },
#         zip_path
#     )

#     n = ingest_from_zip("climate_test", zip_path)
#     print(f"   ✔ Insertadas/actualizadas {n} filas ({year}-{month})")

# conn.close()


In [1]:
import os
import sqlite3
import pandas as pd
import cdsapi
import zipfile
from netCDF4 import Dataset, num2date

# ————— Configuración de prototipo —————
DB_PATH   = r"C:\Users\Nainh\Proton Drive\nainho1306\My files\Licitacion\Plataforma local\backend\data\mi_base_de_datos5.db"
TMP_DIR   = "tmp_test20d"
CDS_URL   = 'https://cds.climate.copernicus.eu/api'
CDS_KEY   = 'd9af180c-f7f8-4a2e-8029-09018fb8c920'
AREA      = [-32.0, -63.0, -33.0, -61.0]  # [Norte, Oeste, Sur, Este]
VARIABLES = ['2m_temperature','2m_dewpoint_temperature','total_precipitation']

os.makedirs(TMP_DIR, exist_ok=True)

# ————— Prepara SQLite y tabla de prueba —————
conn = sqlite3.connect(DB_PATH)
cur  = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS climate_test (
    date TEXT,
    latitude REAL,
    longitude REAL,
    t2m REAL,
    d2m REAL,
    tp REAL,
    PRIMARY KEY(date, latitude, longitude)
)
""")
conn.commit()

# ————— Calcula últimos 20 días —————
end   = pd.to_datetime("today").normalize() - pd.Timedelta(days=1)
start = end - pd.Timedelta(days=19)
dates = pd.date_range(start, end, freq="D")
years  = sorted({str(d.year)     for d in dates})
months = sorted({f"{d.month:02d}" for d in dates})
days   = sorted({f"{d.day:02d}"   for d in dates})

print(f"Descargando datos de {start.date()} a {end.date()} ({len(dates)} días)…")

# ————— Descarga del ZIP con los 3 *.nc —————
zip_path = os.path.join(TMP_DIR, "test20d.zip")
c = cdsapi.Client(url=CDS_URL, key=CDS_KEY)
c.retrieve(
    'derived-era5-single-levels-daily-statistics',
    {
        'product_type':    'reanalysis',
        'format':          'netcdf',       # el API nos devuelve un ZIP
        'variable':        VARIABLES,
        'year':            years,
        'month':           months,
        'day':             days,
        'daily_statistic': 'daily_mean',
        'frequency':       '6_hourly',
        'time_zone':       'utc+00:00',
        'area':            AREA,
    },
    zip_path
)

import tempfile

def ingest_from_zip(table_name, zip_filepath):
    """Extrae cada .nc a un NamedTemporaryFile y vuelca a SQLite."""
    extracted_paths = []

    # 1) Abre el ZIP
    with zipfile.ZipFile(zip_filepath, 'r') as z:
        nc_members = [m for m in z.namelist() if m.endswith('.nc')]
        if not nc_members:
            raise RuntimeError("No encontré archivos .nc en el ZIP")

        for member in nc_members:
            # crea un temp file garantizado escribible
            tmpf = tempfile.NamedTemporaryFile(suffix=".nc", delete=False)
            # vuelca el contenido
            with z.open(member) as src:
                tmpf.write(src.read())
            tmpf.close()
            extracted_paths.append(tmpf.name)

    # 2) Lee coordenadas y variables
    data_arrays = {}
    time_objs = latitudes = longitudes = None

    for nc_path in extracted_paths:
        ds = Dataset(nc_path)
        # detecta nombre de dim/vars
        if time_objs is None:
            time_var = next(v for v in ds.variables if "time" in v.lower())
            lat_var  = next(v for v in ds.variables if "lat"  in v.lower())
            lon_var  = next(v for v in ds.variables if "lon"  in v.lower())
            times    = ds.variables[time_var][:]
            time_objs= num2date(times,
                        ds.variables[time_var].units,
                        getattr(ds.variables[time_var], 'calendar', 'standard'))
            latitudes = ds.variables[lat_var][:]
            longitudes= ds.variables[lon_var][:]
        # identifica la variable de datos
        var_keys = [v for v in ds.variables 
                    if v not in (time_var, lat_var, lon_var, "number")]
        if len(var_keys) != 1:
            raise RuntimeError(f"Variables inesperadas en {nc_path}: {var_keys}")
        var = var_keys[0]
        data_arrays[var] = ds.variables[var][:]
        ds.close()

    # 3) Inserta en SQLite
    sql = f"""INSERT OR REPLACE INTO {table_name}
              (date, latitude, longitude, t2m, d2m, tp)
              VALUES (?, ?, ?, ?, ?, ?)"""
    cnt = 0
    for ti, dt in enumerate(time_objs):
        # usa strftime directamente en el objeto cftime
        date_str = dt.strftime("%Y-%m-%d")
        for yi, lat in enumerate(latitudes):
            for xi, lon in enumerate(longitudes):
                v_t2m = float(data_arrays.get("t2m", data_arrays.get("2m_temperature"))[ti,yi,xi])
                v_d2m = float(data_arrays.get("d2m", data_arrays.get("2m_dewpoint_temperature"))[ti,yi,xi])
                v_tp  = float(data_arrays.get("tp",  data_arrays.get("total_precipitation"))[ti,yi,xi])
                cur.execute(sql, (date_str, float(lat), float(lon), v_t2m, v_d2m, v_tp))
                cnt += 1
    conn.commit()

    # 4) Borra los temporales
    for p in extracted_paths:
        try:
            os.remove(p)
        except OSError:
            pass
    try:
        os.remove(zip_filepath)
    except OSError:
        pass

    return cnt


# ————— Ejecuta ingesta y reporta —————
inserted = ingest_from_zip("climate_test", zip_path)
print(f"✔ Insertadas/actualizadas {inserted} filas en climate_test")

conn.close()


Descargando datos de 2025-07-01 a 2025-07-20 (20 días)…


2025-07-21 16:32:23,298 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-21 16:32:24,174 INFO Request ID is 4eed9029-0fb8-43da-8d04-8eafa0fa45a9
2025-07-21 16:32:24,259 INFO status has been updated to accepted
2025-07-21 16:32:38,312 INFO status has been updated to running
2025-07-21 16:33:40,344 INFO status has been updated to successful


adfff12129898cc4ca364436f4cf98bb.zip:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

✔ Insertadas/actualizadas 675 filas en climate_test


In [3]:
# import os
# import sqlite3
# import pandas as pd
# import cdsapi
# import zipfile
# import tempfile
# from netCDF4 import Dataset, num2date

# # ————— Configuración —————
# DB_PATH   = r"C:\Users\Nainh\Proton Drive\nainho1306\My files\Licitacion\Plataforma local\backend\data\mi_base_de_datos5.db"
# TMP_DIR   = "tmp_test_batches"
# CDS_URL   = 'https://cds.climate.copernicus.eu/api'
# CDS_KEY   = 'd9af180c-f7f8-4a2e-8029-09018fb8c920'
# AREA      = [-32.0, -63.0, -33.0, -61.0]
# VARIABLES = ['2m_temperature','2m_dewpoint_temperature','total_precipitation']

# os.makedirs(TMP_DIR, exist_ok=True)

# # ————— Conexión y tabla —————
# conn = sqlite3.connect(DB_PATH)
# cur  = conn.cursor()
# cur.execute("""
# CREATE TABLE IF NOT EXISTS climate_test (
#     date TEXT,
#     latitude REAL, longitude REAL,
#     t2m REAL, d2m REAL, tp REAL,
#     PRIMARY KEY(date, latitude, longitude)
# )
# """)
# conn.commit()

# # ————— Fechas de 2025-01-01 a ayer —————
# start = pd.Timestamp("2025-01-01")
# end   = pd.to_datetime("today").normalize() - pd.Timedelta(days=1)
# dates = pd.date_range(start, end, freq="D")

# # Agrupamos por mes y ordenamos
# periods = sorted(dates.to_series().dt.to_period('M').unique())
# total = len(periods)

# print(f"Procesando {total} meses, de {periods[0]} a {periods[-1]}…")

# # ————— Función de ingesta —————
# def ingest_from_zip(table_name, zip_filepath):
#     extracted = []
#     with zipfile.ZipFile(zip_filepath, 'r') as z:
#         for member in z.namelist():
#             if not member.endswith('.nc'): continue
#             tmpf = tempfile.NamedTemporaryFile(suffix=".nc", delete=False)
#             tmpf.write(z.read(member))
#             tmpf.close()
#             extracted.append(tmpf.name)

#     data = {}; times = lats = lons = None
#     for path in extracted:
#         ds = Dataset(path)
#         if times is None:
#             tvar = next(v for v in ds.variables if "time" in v.lower())
#             latv = next(v for v in ds.variables if "lat"  in v.lower())
#             lonv = next(v for v in ds.variables if "lon"  in v.lower())
#             times = num2date(ds.variables[tvar][:],
#                              ds.variables[tvar].units,
#                              getattr(ds.variables[tvar],'calendar','standard'))
#             lats  = ds.variables[latv][:]
#             lons  = ds.variables[lonv][:]
#         var_keys = [v for v in ds.variables if v not in (tvar, latv, lonv, "number")]
#         data[var_keys[0]] = ds.variables[var_keys[0]][:]
#         ds.close()

#     sql = f"""INSERT OR REPLACE INTO {table_name}
#               (date, latitude, longitude, t2m, d2m, tp)
#               VALUES (?, ?, ?, ?, ?, ?)"""
#     cnt = 0
#     for ti, dt in enumerate(times):
#         dstr = dt.strftime("%Y-%m-%d")
#         for yi, lat in enumerate(lats):
#             for xi, lon in enumerate(lons):
#                 cur.execute(sql, (
#                     dstr, float(lat), float(lon),
#                     float(data.get('2m_temperature', data.get('t2m'))[ti,yi,xi]),
#                     float(data.get('2m_dewpoint_temperature', data.get('d2m'))[ti,yi,xi]),
#                     float(data.get('total_precipitation', data.get('tp'))[ti,yi,xi])
#                 ))
#                 cnt += 1
#     conn.commit()
#     for fpath in extracted + [zip_filepath]:
#         try: os.remove(fpath)
#         except: pass
#     return cnt

# # ————— Descarga e ingesta por mes con progreso —————
# c = cdsapi.Client(url=CDS_URL, key=CDS_KEY)
# for idx, period in enumerate(periods, start=1):
#     year  = str(period.year)
#     month = f"{period.month:02d}"
#     # Filtrar sólo las fechas de ese mes
#     days  = [f"{d.day:02d}" for d in dates if d.to_period('M') == period]

#     zip_path = os.path.join(TMP_DIR, f"{year}{month}.zip")
#     print(f"[{idx}/{total}] {year}-{month}: días {days}")

#     # Descarga
#     c.retrieve(
#         'derived-era5-single-levels-daily-statistics',
#         {
#             'product_type':    'reanalysis',
#             'format':          'netcdf',
#             'variable':        VARIABLES,
#             'year':            [year],
#             'month':           [month],
#             'day':             days,
#             'daily_statistic': 'daily_mean',
#             'frequency':       '6_hourly',
#             'time_zone':       'utc+00:00',
#             'area':            AREA,
#         },
#         zip_path
#     )

#     # Ingesta
#     inserted = ingest_from_zip("climate_test", zip_path)
#     pct = idx/total*100
#     print(f"   ✔ Insertadas {inserted} filas. Progreso: {pct:.1f}%")

# conn.close()


In [4]:
import winsound

# Beep sencillo: frecuencia 1 kHz durante 500 ms
winsound.Beep(1000, 1300)

In [5]:
import sqlite3
import pandas as pd

# 1) Abre la conexión
conn = sqlite3.connect(DB_PATH)

# 2) Lée los últimos 20 registros (ordenados por fecha decreciente)
df = pd.read_sql_query("""
    SELECT *
      FROM climate_test
     ORDER BY date DESC, latitude DESC, longitude DESC
     LIMIT 20
""", conn)

# 3) Muestra el DataFrame
print(df)

# 4) Cierra la conexión
conn.close()


          date  latitude  longitude         t2m         d2m   tp t2m_min  \
0   2025-07-02    -32.00     -61.00  277.063599  271.632050  0.0    None   
1   2025-07-02    -32.00     -61.25  276.032837  270.251190  0.0    None   
2   2025-07-02    -32.00     -61.50  275.775513  269.354218  0.0    None   
3   2025-07-02    -32.00     -61.75  276.478638  268.784882  0.0    None   
4   2025-07-02    -32.00     -62.00  276.754028  268.600800  0.0    None   
5   2025-07-02    -32.00     -62.25  276.935669  268.486053  0.0    None   
6   2025-07-02    -32.00     -62.50  277.075806  268.494354  0.0    None   
7   2025-07-02    -32.00     -62.75  277.171997  269.055389  0.0    None   
8   2025-07-02    -32.00     -63.00  277.255005  269.951874  0.0    None   
9   2025-07-02    -32.25     -61.00  276.527466  271.567596  0.0    None   
10  2025-07-02    -32.25     -61.25  275.487915  270.188202  0.0    None   
11  2025-07-02    -32.25     -61.50  275.347778  269.501190  0.0    None   
12  2025-07-

In [4]:
import os
import sqlite3
import pandas as pd
import cdsapi
import zipfile
import tempfile
from netCDF4 import Dataset, num2date

# ————— Configuración —————
DB_PATH   = r"C:\Users\Nainh\Proton Drive\nainho1306\My files\Licitacion\Plataforma local\backend\data\mi_base_de_datos5.db"
TMP_DIR   = "tmp_clima_nov_dic_2024"
CDS_URL   = 'https://cds.climate.copernicus.eu/api'
CDS_KEY   = 'd9af180c-f7f8-4a2e-8029-09018fb8c920'
AREA      = [-32.0, -63.0, -33.0, -61.0]
VARIABLES = ['2m_temperature','2m_dewpoint_temperature','total_precipitation']

os.makedirs(TMP_DIR, exist_ok=True)

# ————— Conecta y asegura la tabla —————
conn = sqlite3.connect(DB_PATH)
cur  = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS climate_test (
    date TEXT,
    latitude REAL, longitude REAL,
    t2m REAL, d2m REAL, tp REAL,
    PRIMARY KEY(date, latitude, longitude)
)
""")
conn.commit()

# ————— Rango fijo de fechas —————
start = pd.Timestamp("2025-07-02")
end   = pd.Timestamp("2025-07-11")
dates = pd.date_range(start, end, freq="D")

# Agrupamos y ordenamos los meses
periods = sorted(dates.to_series().dt.to_period('M').unique())
total = len(periods)
print(f"Descargando datos desde {start.date()} hasta {end.date()} → {total} meses: {periods}")

# ————— Función de ingesta —————
def ingest_from_zip(table, zip_fp):
    tmp_files = []
    with zipfile.ZipFile(zip_fp) as z:
        for member in z.namelist():
            if not member.endswith('.nc'): continue
            tmp = tempfile.NamedTemporaryFile(suffix=".nc", delete=False)
            tmp.write(z.read(member)); tmp.close()
            tmp_files.append(tmp.name)

    data, times, lats, lons = {}, None, None, None
    for nc in tmp_files:
        ds = Dataset(nc)
        if times is None:
            tvar = next(v for v in ds.variables if "time" in v.lower())
            latv = next(v for v in ds.variables if "lat"  in v.lower())
            lonv = next(v for v in ds.variables if "lon"  in v.lower())
            times = num2date(ds.variables[tvar][:],
                             ds.variables[tvar].units,
                             getattr(ds.variables[tvar],'calendar','standard'))
            lats  = ds.variables[latv][:]
            lons  = ds.variables[lonv][:]
        var_keys = [v for v in ds.variables if v not in (tvar, latv, lonv, "number")]
        data[var_keys[0]] = ds.variables[var_keys[0]][:]
        ds.close()

    sql = f"""INSERT OR REPLACE INTO {table}
              (date,latitude,longitude,t2m,d2m,tp)
              VALUES (?,?,?,?,?,?)"""
    count = 0
    for ti, dt in enumerate(times):
        # Aquí usamos directamente strftime sobre el objeto cftime
        day = dt.strftime("%Y-%m-%d")
        for yi, lat in enumerate(lats):
            for xi, lon in enumerate(lons):
                cur.execute(sql, (
                    day, float(lat), float(lon),
                    float(data.get('2m_temperature', data.get('t2m'))[ti,yi,xi]),
                    float(data.get('2m_dewpoint_temperature', data.get('d2m'))[ti,yi,xi]),
                    float(data.get('total_precipitation', data.get('tp'))[ti,yi,xi])
                ))
                count += 1
    conn.commit()

    # Limpieza
    for f in tmp_files + [zip_fp]:
        try: os.remove(f)
        except: pass

    return count

# ————— Batch mensual con progreso —————
client = cdsapi.Client(url=CDS_URL, key=CDS_KEY)
for idx, period in enumerate(periods, start=1):
    year  = str(period.year)
    month = f"{period.month:02d}"
    days  = [f"{d.day:02d}" for d in dates if d.to_period('M') == period]

    zip_path = os.path.join(TMP_DIR, f"{year}{month}.zip")
    print(f"[{idx}/{total}] Descargando {year}-{month}: días {days} …")

    client.retrieve(
        'derived-era5-single-levels-daily-statistics',
        {
            'product_type':    'reanalysis',
            'format':          'netcdf',
            'variable':        VARIABLES,
            'year':            [year],
            'month':           [month],
            'day':             days,
            'daily_statistic': 'daily_mean',
            'frequency':       '6_hourly',
            'time_zone':       'utc+00:00',
            'area':            AREA,
        },
        zip_path
    )

    inserted = ingest_from_zip("climate_test", zip_path)
    pct = idx/total*100
    print(f"   ✔ Insertadas {inserted} filas — Progreso: {pct:.1f}%")

conn.close()


2025-07-11 16:19:55,568 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.


Descargando datos desde 2025-07-02 hasta 2025-07-11 → 1 meses: [Period('2025-07', 'M')]
[1/1] Descargando 2025-07: días ['02', '03', '04', '05', '06', '07', '08', '09', '10', '11'] …


2025-07-11 16:19:56,140 INFO Request ID is 632dee09-9fb6-4e70-9ce0-f96f84a479fe
2025-07-11 16:19:56,209 INFO status has been updated to accepted
2025-07-11 16:20:04,627 INFO status has been updated to running
2025-07-11 16:20:17,516 INFO status has been updated to successful


   ✔ Insertadas 180 filas — Progreso: 100.0%


In [ ]:
import os
import sqlite3
import pandas as pd
import cdsapi
import zipfile
import tempfile
from netCDF4 import Dataset, num2date

# ————— Configuración —————
DB_PATH   = r"C:\Users\Nainh\Proton Drive\nainho1306\My files\Licitacion\Plataforma local\backend\data\mi_base_de_datos5.db"
TMP_DIR   = "tmp_test20d"
CDS_URL   = 'https://cds.climate.copernicus.eu/api'
CDS_KEY   = 'd9af180c-f7f8-4a2e-8029-09018fb8c920'
AREA      = [-32.0, -63.0, -33.0, -61.0]  # [Norte, Oeste, Sur, Este]
VARIABLES = ['2m_temperature','2m_dewpoint_temperature','total_precipitation']

# Variables y columnas destino por estadística
STATS = {
    'mínima':  ('daily_min', ['t2m_min']),
    'máxima':  ('daily_max', ['t2m_max']),
    'viento':  ('daily_mean', ['10m_u_component_of_wind',
                               '10m_v_component_of_wind'])
}

os.makedirs(TMP_DIR, exist_ok=True)
conn = sqlite3.connect(DB_PATH)
cur  = conn.cursor()

# Asegúrate de tablas y columnas
cur.execute("""
CREATE TABLE IF NOT EXISTS climate_stats (
    date TEXT, latitude REAL, longitude REAL,
    t2m_min REAL, t2m_max REAL,
    u10_mean REAL, v10_mean REAL, wind_speed REAL,
    PRIMARY KEY(date, latitude, longitude)
)""")
conn.commit()

def ingest_zip(table, file_fp, vars_nc, cols):
    """Extrae .nc del ZIP, lee con netCDF4 y vuelca cols en la tabla."""
    # 1) Extraer .nc(s)
    paths = []
    if zipfile.is_zipfile(file_fp):
        with zipfile.ZipFile(file_fp, 'r') as z:
            members = [m for m in z.namelist() if m.endswith('.nc')]
            if not members:
                raise RuntimeError(f"ZIP {file_fp} no contiene .nc")
            for m in members:
                tmp = tempfile.NamedTemporaryFile(suffix=".nc", delete=False)
                tmp.write(z.read(m))
                tmp.close()
                paths.append(tmp.name)
    else:
        # 2) No es ZIP: leemos magic-bytes para ver si HDF5/netCDF3
        with open(file_fp, 'rb') as f:
            hb = f.read(8)
        if hb.startswith(b'\211HDF') or b'CDF' in hb:
            paths = [file_fp]
        else:
            raise RuntimeError(
                f"{file_fp} no es ZIP ni NetCDF válido, primeros bytes: {hb!r}"
            )

    # 3) Leer coords/times y cada variable
    data = {}
    times = lats = lons = None
    for p in paths:
        ds = Dataset(p)
        if times is None:
            tvar = next(v for v in ds.variables if "time" in v.lower())
            latv = next(v for v in ds.variables if "lat"  in v.lower())
            lonv = next(v for v in ds.variables if "lon"  in v.lower())
            times = num2date(ds.variables[tvar][:],
                             ds.variables[tvar].units,
                             getattr(ds.variables[tvar],'calendar','standard'))
            lats = ds.variables[latv][:]
            lons = ds.variables[lonv][:]
        for var, col in zip(vars_nc, cols):
            data[col] = ds.variables[var][:]
        ds.close()

    # 4) Insert/upsert
    cols_sql = ",".join(cols)
    vals_sql = ",".join("?" for _ in cols)
    sql = f"""
      INSERT INTO {table}
        (date, latitude, longitude, {cols_sql})
      VALUES (?,?,?,{vals_sql})
      ON CONFLICT(date,latitude,longitude) DO UPDATE SET
        {", ".join(f"{c}=excluded.{c}" for c in cols)}
    """
    for ti, dt in enumerate(times):
        day = dt.strftime("%Y-%m-%d")
        for yi, lat in enumerate(lats):
            for xi, lon in enumerate(lons):
                row = [day, float(lat), float(lon)] + [float(data[c][ti,yi,xi]) for c in cols]
                cur.execute(sql, row)
    conn.commit()

    # 5) limpiar temporales
    for p in paths + [file_fp]:
        try:
            os.remove(p)
        except OSError:
            pass

# ————— Bucle principal —————
client = cdsapi.Client(url=CDS_URL, key=CDS_KEY)

# Rango de ejemplo: diciembre 2023–marzo 2024
start = pd.Timestamp("2023-12-01")
end   = pd.Timestamp("2024-03-31")
dates = pd.date_range(start, end, freq="D")
periods = sorted(dates.to_series().dt.to_period('M').unique())
total = len(periods)

for stat_name, (daily_stat, cols) in STATS.items():
    for idx, period in enumerate(periods, start=1):
        y = str(period.year)
        m = f"{period.month:02d}"
        days = [f"{d.day:02d}" for d in dates if d.to_period('M') == period]
        zip_fp = os.path.join(TMP_DIR, f"{y}{m}_{stat_name}.zip")

        print(f"[{idx}/{total}] Descargando {stat_name} {y}-{m} …")
        print(f"  daily_statistic: {daily_stat}, variables: {cols}")

        params = {
            'product_type':    'reanalysis',
            'format':          'netcdf',
            'variable':        cols if stat_name == 'viento' else ['2m_temperature'],
            'year':            [y],
            'month':           [m],
            'day':             days,
            'daily_statistic': [daily_stat],  # ahora en lista
            'frequency':       'daily',
            'area':            AREA
        }

        try:
            client.retrieve(
                'derived-era5-single-levels-daily-statistics',
                params,
                zip_fp
            )
        except Exception as e:
            print("ERROR en retrieve:", e)
            # Si CDS devuelve cuerpo de error JSON, lo mostramos:
            if hasattr(e, 'response') and e.response is not None:
                try:
                    print(e.response.text)
                except Exception:
                    pass
            raise

        ingest_zip(
            "climate_stats",
            zip_fp,
            cols if stat_name != 'viento' else
                ['10m_u_component_of_wind', '10m_v_component_of_wind'],
            cols
        )
        print(f"   ✔ {stat_name} {y}-{m} insertado")

conn.close()


2025-07-07 15:47:05,470 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.


[1/4] Descargando mínima 2023-12 …
  daily_statistic: daily_min, variables: ['t2m_min']


2025-07-07 15:47:06,071 INFO Request ID is fcec75e3-f11f-464c-b8bc-6a7cbbbc67ff
2025-07-07 15:47:06,215 INFO status has been updated to accepted
2025-07-07 15:47:14,683 INFO status has been updated to running
2025-07-07 15:47:19,811 INFO status has been updated to failed


ERROR en retrieve: 400 Client Error: Bad Request for url: https://cds.climate.copernicus.eu/api/retrieve/v1/jobs/fcec75e3-f11f-464c-b8bc-6a7cbbbc67ff/results
The job has failed.
The job failed with: TypeError
{"type":"job results failed","title":"The job has failed.","status":400,"instance":"https://cds.climate.copernicus.eu/api/retrieve/v1/jobs/fcec75e3-f11f-464c-b8bc-6a7cbbbc67ff/results","trace_id":"46a14de6-862c-4642-980f-ffa6f845136e","traceback":"The job failed with: TypeError"}


HTTPError: 400 Client Error: Bad Request for url: https://cds.climate.copernicus.eu/api/retrieve/v1/jobs/fcec75e3-f11f-464c-b8bc-6a7cbbbc67ff/results
The job has failed.
The job failed with: TypeError

In [ ]:
import pandas as pd
import cdsapi, xarray as xr, os

# tu DataFrame original
df = pd.read_csv("puntos.csv", parse_dates=["Fechas notificados"])
df["lat"] = df["lat"].astype(float)
df["lon"] = df["lon"].astype(float)

# configura CDS


CDS_URL   = 'https://cds.climate.copernicus.eu/api'
CDS_KEY   = 'd9af180c-f7f8-4a2e-8029-09018fb8c920'

c = cdsapi.Client(url=CDS_URL, key=CDS_KEY)



TMP = "tmp_pts"
os.makedirs(TMP, exist_ok=True)

# constante semicírculo de celda ERA5
HALF = 0.125

# 1) recorre cada par único (lon,lat)
for (lon, lat), sub in df.groupby(["lon","lat"]):
    # agrupa por mes
    sub["mes"] = sub["Fechas notificados"].dt.to_period("M")
    for period, chunk in sub.groupby("mes"):
        yyyy = str(period.year)
        mm   = f"{period.month:02d}"
        days = sorted(chunk["Fechas notificados"]
                      .dt.day
                      .astype(str)
                      .str.zfill(2)
                      .unique().tolist())
        # define área 1×1
        area = [lat+HALF, lon-HALF, lat-HALF, lon+HALF]
        out  = os.path.join(TMP, f"{yyyy}{mm}_{lat}_{lon}.nc")
        # 2) descarga puntual
        c.retrieve(
          "derived-era5-single-levels-daily-statistics",
          {
            "product_type":    "reanalysis",
            "format":          "netcdf",
            "variable":        ["2m_temperature"],
            "year":            [yyyy],
            "month":           [mm],
            "day":             days,
            "daily_statistic": "daily_mean",
            "frequency":       "1_hourly",
            "time_zone":       "utc+00:00",
            "area":            area,
          },
          out
        )
        # 3) asigna valores al df
        ds = xr.open_dataset(out)
        for _, row in chunk.iterrows():
            t = row["Fechas notificados"].strftime("%Y-%m-%d")
            val = ds["t2m"].sel(
                    latitude  = lat,
                    longitude = lon,
                    time      = t,
                    method    = "nearest"
                  ).item()
            df.loc[row.name, "t2m"] = val
        ds.close()
        os.remove(out)

print(df)



2025-06-26 13:40:16,384 WARNING [2025-06-23T00:00:00] Scheduled System Session affecting Service reliability - 30 June 2025. Please follow status [here](https://status.ecmwf.int/) or in our [forum](https://forum.ecmwf.int/t/scheduled-maintenance-of-the-cloud-infrastructure-on-30-june-2025/13598)
2025-06-26 13:40:16,385 INFO [2025-06-16T00:00:00] CC-BY licence to replace Licence to use Copernicus Products on 02 July 2025. More information available [here](https://forum.ecmwf.int/t/cc-by-licence-to-replace-licence-to-use-copernicus-products-on-02-july-2025/13464)
2025-06-26 13:40:16,386 INFO [2025-06-10T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-06-26 13:40:16,387 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-06-26 13:40:17,079 INFO Request ID is cdd49973-ffec-4335-bc46-42bedcf6cdb4
2025-06

PermissionError: [WinError 32] El proceso no tiene acceso al archivo porque está siendo utilizado por otro proceso: 'tmp_pts\\202503_-31.75_-64.5.nc'

In [ ]:
import cdsapi
c = cdsapi.Client(url=CDS_URL, key=CDS_KEY)
dataset = "derived-era5-single-levels-daily-statistics"
request = {
    "product_type": "reanalysis",
    "variable": ["2m_temperature"],
    "year": "2023",
    "month": ["12"],
    "day": ["02", "03", "04"],
    "daily_statistic": "daily_minimum",
    "time_zone": "utc+00:00",
    "frequency": "1_hourly",
    "area": [-32, -63, -33, -61]
}


c.retrieve(
  'derived-era5-single-levels-daily-statistics',
  {
    'product_type':    'reanalysis',
    'format':          'netcdf',
    'variable':        ['2m_temperature'],
    'year':            ['2025'],
    'month':           ['02'],
    'day':             ['01','02','03'],
    'daily_statistic': 'daily_min',     # mínima diaria
    'frequency':       '1_hourly',      # muestreo cada hora
    'time_zone':       'utc+00:00',
    'area':            AREA,
  },
  zip_path
)



2025-06-25 16:05:37,750 WARNING [2025-06-23T00:00:00] Scheduled System Session affecting Service reliability - 30 June 2025. Please follow status [here](https://status.ecmwf.int/) or in our [forum](https://forum.ecmwf.int/t/scheduled-maintenance-of-the-cloud-infrastructure-on-30-june-2025/13598)
2025-06-25 16:05:37,751 INFO [2025-06-16T00:00:00] CC-BY licence to replace Licence to use Copernicus Products on 02 July 2025. More information available [here](https://forum.ecmwf.int/t/cc-by-licence-to-replace-licence-to-use-copernicus-products-on-02-july-2025/13464)
2025-06-25 16:05:37,751 INFO [2025-06-10T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-06-25 16:05:37,752 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-06-25 16:05:39,694 INFO Request ID is afb3b3ff-1b48-44e9-b3c6-cf8ed05b7912
2025-06

'tmp_test20d\\test20d.zip'

In [ ]:
from netCDF4 import Dataset

# Abre el archivo (modo lectura por defecto)
ds = Dataset("out.nc", "r")

# Explora sus dimensiones
print(ds.dimensions)        
# Explora sus variables
print(ds.variables.keys())  

# Accede a una variable concreta, por ejemplo “t2m”
t2m = ds.variables["t2m"][:]   # carga todos sus datos en memoria

# Metadatos generales
print(ds)          # cabeceras, atributos globales
print(t2m.units)   # atributos de la variable

ds.close()

{'valid_time': "<class 'netCDF4.Dimension'>": name = 'valid_time', size = 3, 'latitude': "<class 'netCDF4.Dimension'>": name = 'latitude', size = 5, 'longitude': "<class 'netCDF4.Dimension'>": name = 'longitude', size = 9}
dict_keys(['t2m', 'number', 'latitude', 'longitude', 'valid_time'])
<class 'netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    GRIB_centre: ecmf
    GRIB_centreDescription: European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre: 0
    Conventions: CF-1.7
    institution: European Centre for Medium-Range Weather Forecasts
    history: 2025-06-25T13:57 GRIB to CDM+CF via cfgrib-0.9.15.0/ecCodes-2.41.0 with {"source": "2m_temperature.grib", "filter_by_keys": {"stream": ["oper"]}, "encode_cf": ["parameter", "time", "geography", "vertical"]}
earthkit.transforms.aggregate.temporal.daily_reduce(2m_temperature_stream-oper, how=min, **{'time_shift': {'hours': 0}})
    dimensions(sizes): valid_time(3), latitude(5), longitude(9)
    variable

AttributeError: 'MaskedArray' object has no attribute 'units'

In [2]:
import os
import pandas as pd
import cdsapi
import xarray as xr
import sqlite3

# ————— Configuración —————
DB_PATH = r"C:\Users\Nainh\Proton Drive\nainho1306\My files\Licitacion\Plataforma local\backend\data\mi_base_de_datos5.db"
TMP_DIR = "tmp_nc_batches"
CDS_URL = 'https://cds.climate.copernicus.eu/api'
CDS_KEY = 'd9af180c-f7f8-4a2e-8029-09018fb8c920'
AREA    = [-32.0, -63.0, -33.0, -61.0]  # [Norte, Oeste, Sur, Este]

os.makedirs(TMP_DIR, exist_ok=True)

# Estadísticas a descargar: (sufijo en el NC, daily_statistic, variable API, frecuencia)
STATS = [
    ("t2m_min",  "daily_min",  "2m_temperature",         "1_hourly"),
    ("t2m_max",  "daily_max",  "2m_temperature",         "1_hourly"),
    ("d2m_mean", "daily_mean", "2m_dewpoint_temperature","6_hourly"),
    ("tp_sum",   "daily_sum",  "total_precipitation",    "6_hourly"),
]

# Rango: desde 2025-01-01 hasta ayer
start   = pd.Timestamp("2025-07-02")
end     = pd.Timestamp("today").normalize() - pd.Timedelta(days=1)
dates   = pd.date_range(start, end, freq="D")
periods = sorted(dates.to_series().dt.to_period("M").unique())

client = cdsapi.Client(url=CDS_URL, key=CDS_KEY)

def insert_nc_to_sql(nc_path, db_path, table="climate_stats"):
    """
    Inserta o actualiza en SQLite los datos del NetCDF que tenga variables
    diarias combinadas (t2m_min, t2m_max, d2m_mean, tp_sum, etc.).
    Detecta automáticamente el nombre de la dimensión tiempo.
    """
    ds = xr.open_dataset(nc_path, engine="netcdf4")
    vars_sql = list(ds.data_vars)  # ej: ['t2m_min','t2m_max','d2m_mean','tp_sum']

    # Detectar ejes: tomamos las dims de la primera variable
    sample = vars_sql[0]
    dims = ds[sample].dims
    time_dim = next(d for d in dims if d.lower().endswith("time"))
    lat_dim  = next(d for d in dims if "lat"  in d.lower())
    lon_dim  = next(d for d in dims if "lon"  in d.lower())

    times = pd.to_datetime(ds[time_dim].values)
    lats  = ds[lat_dim].values
    lons  = ds[lon_dim].values

    # Conectar y preparar tabla
    conn = sqlite3.connect(db_path)
    cur  = conn.cursor()
    cols_def = ", ".join(f"{v} REAL" for v in vars_sql)
    cur.execute(f"""
      CREATE TABLE IF NOT EXISTS {table} (
        date TEXT,
        latitude REAL, longitude REAL,
        {cols_def},
        PRIMARY KEY(date, latitude, longitude)
      )
    """)
    conn.commit()

    # SQL dinámico: INSERT ... ON CONFLICT DO UPDATE
    cols_sql = ",".join(vars_sql)
    ph       = ",".join("?" for _ in vars_sql)
    sql = f"""
      INSERT INTO {table}
        (date, latitude, longitude, {cols_sql})
      VALUES (?,?,?,{ph})
      ON CONFLICT(date,latitude,longitude) DO UPDATE SET
        {", ".join(f"{v}=excluded.{v}" for v in vars_sql)}
    """

    # Volcar fila a fila
    for ti, dt in enumerate(times):
        date_str = dt.strftime("%Y-%m-%d")
        for yi, lat in enumerate(lats):
            for xi, lon in enumerate(lons):
                row = [date_str, float(lat), float(lon)] + \
                      [float(ds[v].values[ti, yi, xi]) for v in vars_sql]
                cur.execute(sql, row)

    conn.commit()
    conn.close()
    ds.close()

# ————— Bucle principal —————
for period in periods:
    y, m = str(period.year), f"{period.month:02d}"
    days = [f"{d.day:02d}" for d in dates if d.to_period("M") == period]
    print(f"\nProcesando {y}-{m}, días: {days}")

    # 1) Descarga cada estadística en su NC propio
    tmp_files = []
    for suffix, daily_stat, var, freq in STATS:
        fp = os.path.join(TMP_DIR, f"{y}{m}_{suffix}.nc")
        print(f"  ▷ Descargando {suffix}… ", end="", flush=True)
        client.retrieve(
            "derived-era5-single-levels-daily-statistics",
            {
                "product_type":    "reanalysis",
                "format":          "netcdf",
                "variable":        [var],
                "year":            [y],
                "month":           [m],
                "day":             days,
                "daily_statistic": daily_stat,
                "frequency":       freq,
                "time_zone":       "utc+00:00",
                "area":            AREA,
            },
            fp
        )
        print("OK")
        tmp_files.append(fp)

    # 2) Merge dinámico y renombrado implícito
    datasets = []
    for fp in tmp_files:
        ds = xr.open_dataset(fp, engine="netcdf4")
        # detecta la única variable data_var
        data_vars = [v for v in ds.data_vars if v not in ("latitude","longitude") 
                     and not v.lower().endswith("time")]
        if len(data_vars) != 1:
            raise RuntimeError(f"{fp} -> variables inesperadas: {data_vars}")
        # renombrar al sufijo (parte tras el guion bajo en el nombre de fichero)
        suffix = os.path.basename(fp).split("_")[-1].replace(".nc","")
        ds = ds.rename({data_vars[0]: suffix})
        datasets.append(ds)

    combined = xr.merge(datasets)
    combined.attrs.clear()

    # 3) Guardar NC combinado
    out_fp = os.path.join(TMP_DIR, f"{y}{m}_daily_stats.nc")
    print(f"  ▷ Guardando combinado → {out_fp}")
    combined.to_netcdf(out_fp, format="NETCDF4")

    # 4) Insertar en SQLite
    print(f"  ▷ Insertando en SQLite desde {out_fp}")
    insert_nc_to_sql(out_fp, DB_PATH, table="climate_stats")

    # 5) Limpiar temporales individuales
    for fp in tmp_files:
        try: os.remove(fp)
        except: pass
    # (opcional) borrar también el combinado:
    # try: os.remove(out_fp)
    # except: pass

print("\n¡Proceso completado!")


2025-07-11 16:13:53,064 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.



Procesando 2025-07, días: ['02', '03', '04', '05', '06', '07', '08', '09', '10']
  ▷ Descargando t2m_min… 

2025-07-11 16:13:53,924 INFO Request ID is fd4e501f-f66a-415c-9c0f-025c77e39c8d
2025-07-11 16:13:53,989 INFO status has been updated to accepted
2025-07-11 16:14:02,595 INFO status has been updated to running
2025-07-11 16:14:07,730 INFO status has been updated to accepted
2025-07-11 16:14:15,397 INFO status has been updated to successful
                                                                                        

OK
  ▷ Descargando t2m_max… 

2025-07-11 16:14:16,720 INFO Request ID is 7807ae49-8800-497f-81b2-d96c9d5a84be
2025-07-11 16:14:16,775 INFO status has been updated to accepted
2025-07-11 16:14:30,272 INFO status has been updated to running
2025-07-11 16:14:37,946 INFO status has been updated to successful
                                                                                        

...

2025-07-11 16:14:39,166 INFO Request ID is d8d7a176-3b08-4c85-b413-f7754c4234ef
2025-07-11 16:14:39,245 INFO status has been updated to accepted
2025-07-11 16:15:00,480 INFO status has been updated to running
2025-07-11 16:15:11,930 INFO status has been updated to successful
                                                                                        

OK
  ▷ Descargando tp_sum… 

2025-07-11 16:15:13,088 INFO Request ID is 9124e726-b1f8-4c58-a1a8-1f666cbcc30a
2025-07-11 16:15:13,162 INFO status has been updated to accepted
2025-07-11 16:15:21,542 INFO status has been updated to running
2025-07-11 16:15:34,329 INFO status has been updated to successful


OK
  ▷ Guardando combinado → tmp_nc_batches\202507_daily_stats.nc
  ▷ Insertando en SQLite desde tmp_nc_batches\202507_daily_stats.nc


OperationalError: table climate_stats has no column named min

In [3]:
import sqlite3
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("PRAGMA table_info(climate_stats)")
print(cursor.fetchall())
conn.close()


[(0, 'date', 'TEXT', 0, None, 1), (1, 'latitude', 'REAL', 0, None, 2), (2, 'longitude', 'REAL', 0, None, 3), (3, 't2m_min', 'REAL', 0, None, 0), (4, 't2m_max', 'REAL', 0, None, 0), (5, 'u10_mean', 'REAL', 0, None, 0), (6, 'v10_mean', 'REAL', 0, None, 0), (7, 'wind_speed', 'REAL', 0, None, 0)]


In [ ]:
CDS_KEY

'd9af180c-f7f8-4a2e-8029-09018fb8c920'

In [ ]:


conn.close()